<a href="https://colab.research.google.com/github/ddekun/PyTorch_framework/blob/lesson10/lesson10/hw10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Фреймворк PyTorch для разработки искусственных нейронных сетей

### Урок 10. Распознавание лиц и эмоций

Задание по итогам курса:
(упрощенное/для тех, у кого нет вебкамеры)

Нужно написать приложение, которое будет получать на вход изображение.
В процессе определять, что перед камерой находится человек, задетектировав его лицо на кадре.
На изображении человек показывает жесты руками, а алгоритм должен считать их и классифицировать.
(более сложное)

Нужно написать приложение, которое будет считывать и выводить кадры с веб-камеры.
В процессе считывания определять что перед камерой находится человек, задетектировав его лицо на кадре.
Человек показывает жесты руками, а алгоритм должен считать их и классифицировать.
Для распознавания жестов, вам надо будет скачать датасет https://www.kaggle.com/gti-upm/leapgestrecog, разработать модель для обучения и обучить эту модель.
Как работать с веб-камерой на google colab https://stackoverflow.com/questions/54389727/opening-web-camera-in-google-colab
У кого нет возможности работать через каггл (нет верификации), то можете данные взять по ссылке: https://disk.yandex.ru/d/R2PGlaXDf6_HzQ

Данная промежуточная аттестация оценивается по системе "зачет" / "не зачет"
Зачет" ставится, если слушатель успешно выполнил задание 1 или 2 задания
"Незачет"" ставится, если слушатель не выплнил 0 заданий

Критерии оценивания:
Слушатель написал приложение, которое будет получать на вход изображение
Слушатель написать приложение, которое будет считывать и выводить кадры с веб-камеры

In [1]:
import os
import time
import glob

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as tt
from torchvision.utils import make_grid
from torchvision.datasets import ImageFolder
from torchvision import datasets, transforms

from sklearn.model_selection import train_test_split

from PIL import Image
from torch.autograd import Variable

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [3]:
CATEGORIES = ['01_palm', '02_l','03_fist','04_fist_moved','05_thumb','06_index','07_ok','08_palm_moved','09_c','10_down']

In [4]:
images = glob.glob('./leapGestRecog/**/**/**/*.png')

label = [int(os.path.basename(i).split('_')[2])-1 for i in images]

In [5]:
train_transformer = transforms.Compose([
                    transforms.Resize((48, 128)),  # пропорциональное изменение от первоначального размера 240х640
                    transforms.RandomHorizontalFlip(),
                    transforms.ToTensor()])

test_transformer = transforms.Compose([
                   transforms.Resize((48, 128)),
                   transforms.Grayscale(num_output_channels=1),
                   transforms.ToTensor()])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(images, label, test_size = 0.3, random_state=42)

In [15]:
class LeapGestRecogDataset(Dataset):
    def __init__(self, filenames, labels, transform):
        self.filenames = filenames
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.filenames)

    def __getitem__(self, idx):
        image = Image.open(self.filenames[idx])
        image = self.transform(image)
        return image, self.labels[idx]

In [ ]:
train_dataset = LeapGestRecogDataset(X_train, y_train, train_transformer)
test_dataset = LeapGestRecogDataset(X_test, y_test, test_transformer)

In [ ]:
batch_size = 32
lr = 0.01
train_dl = DataLoader(train_dataset, batch_size, shuffle=True)
test_dl = DataLoader(test_dataset, batch_size)

In [ ]:
data_iter = iter(train_dl)

img, labels = data_iter.next()

plt.figure(figsize=(14, 21))
for i in range(batch_size):
    ax = plt.subplot(9, 3, i+1)
    plt.imshow(img[i, 0, :, :], cmap='gray')
    plt.title(f'{CATEGORIES[int(labels[i])][3:]}')
    if i > 10:
        break

In [18]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 32, 3, 1)
        self.dropout1 = nn.Dropout2d(0.3)
        self.fc1 = nn.Linear(32*22*62, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

In [ ]:
model = Net().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lr)

In [20]:
epochs = 10

In [21]:
def fit(model, epochs):

    epoch_losses = []
    epoch_acc = []
    epoch_val_losses = []
    epoch_val_acc = []
    bundle = 20

    for epoch in range(epochs):
        running_loss = 0.0
        cum_loss = []
        cum_acc = []

        time1 = time.time()
        model.train()
        for i, data in enumerate(train_dl, 0):
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            cum_loss.append(loss.item())

            total = labels.size(0)
            _, predicted = torch.max(outputs.data, 1)
            correct = (predicted == labels).sum().item()
            cum_acc.append(correct / total)

        time2 = time.time()
        epoch_losses.append(np.mean(cum_loss))
        epoch_acc.append(np.mean(cum_acc))

        cum_loss = []
        cum_acc = []
        model.eval()
        with torch.no_grad():
            running_val_loss = 0.0
            for i, data in enumerate(test_dl, 0):
                inputs, labels = data
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                running_val_loss += loss.item()
                cum_loss.append(loss.item())

                total = labels.size(0)
                _, predicted = torch.max(outputs.data, 1)
                correct = (predicted == labels).sum().item()
                cum_acc.append(correct / total)

        epoch_val_losses.append(np.mean(cum_loss))
        epoch_val_acc.append(np.mean(cum_acc))
        print(f'Epochs: {epoch+1}   loss: {epoch_losses[-1]:.5f}   '
              f'accuracy: {epoch_acc[-1]*100:.3f}%   '
              f'loss_val: {epoch_val_losses[-1]:.5f}   '
              f'accuracy_val: {epoch_val_acc[-1]*100:.3f}%   '
              f'time = {time2-time1:.2f} s')

    return {'epoch_losses': epoch_losses,
            'epoch_val_losses': epoch_val_losses,
            'epoch_acc': epoch_acc,
            'epoch_val_acc': epoch_val_acc}

In [ ]:
history = fit(model, epochs)

In [ ]:
torch.save(model.state_dict(), './leapGestRecog/gesture_recognition')

In [ ]:
net = Net().to(device)
net.load_state_dict(torch.load('./leapGestRecog/gesture_recognition', map_location=torch.device('cpu')))
net.eval()

In [ ]:
data_iter = iter(test_dl)

img, labels = data_iter.next()

model.eval()
with torch.no_grad():
    inputs = img.to(device)
    labels = labels.to(device)

    outputs = net(inputs)
    loss = criterion(outputs, labels)

    total = labels.size(0)
    _, predicted = torch.max(outputs.data, 1)
    correct = (predicted == labels).sum().item()
    print(f'Correct: {correct} / {batch_size}')

plt.figure(figsize=(14, 21))
for i in range(batch_size):
    ax = plt.subplot(9, 3, i+1)

    plt.imshow(img[i, 0, :, :], cmap='gray')
    if int(labels[i]) == int(predicted[i]):
        plt.title(f'{CATEGORIES[int(labels[i])][3:]} / {CATEGORIES[int(predicted[i])][3:]}')
    else:
        plt.title(f'Error: {CATEGORIES[int(labels[i])][3:]} / {CATEGORIES[int(predicted[i])][3:]}')
    plt.axis('off')
    if i > 10:
        break